# Fine Tuning Flan T5

References:

1. https://www.datacamp.com/tutorial/flan-t5-tutorial
2. https://www.youtube.com/watch?v=r6XY80Z9eSA (Ignore Lightning, this also uses trainer instead of seq2seq trainer)
3. https://discuss.huggingface.co/t/trainer-vs-seq2seqtrainer/3145 (Why I'm using seq2seq trainer instead of regular trainer)

In [7]:
DATASET_FILE_PATH= 'temp/zillow_qa_dataset.json'

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import nltk
import evaluate
import numpy as np
from datasets import load_dataset

dataset = load_dataset('json', data_files=DATASET_FILE_PATH)

dataset = dataset['train'].train_test_split(test_size=0.2)

dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 200
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 50
    })
})

In [9]:
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

def preprocess_function(examples):
    inputs = [q for q in examples['question']]
    targets = [a for a in examples['answer']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

tokenized_dataset

c:\Users\kevin\mads-capstone-project\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kevin\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-t

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
})

In [10]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result


In [11]:
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

In [12]:
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)


trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)


trainer.train()


c:\Users\kevin\mads-capstone-project\.venv\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(














                                            

                                                 
  4%|▍         | 3/75 [20:06<4:27:40, 223.06s/it]


{'eval_loss': 0.4759933650493622, 'eval_rouge1': 0.8030727969348662, 'eval_rouge2': 0.7355647130647129, 'eval_rougeL': 0.7945818281335524, 'eval_rougeLsum': 0.7940235358511223, 'eval_runtime': 35.3048, 'eval_samples_per_second': 1.416, 'eval_steps_per_second': 0.368, 'epoch': 1.0}
















                                            

                                                 
  4%|▍         | 3/75 [22:39<4:27:40, 223.06s/it]


{'eval_loss': 0.35776078701019287, 'eval_rouge1': 0.7983442802408318, 'eval_rouge2': 0.7325657213157213, 'eval_rougeL': 0.7888382594417076, 'eval_rougeLsum': 0.7879885057471263, 'eval_runtime': 35.8781, 'eval_samples_per_second': 1.394, 'eval_steps_per_second': 0.362, 'epoch': 2.0}
















                                                 

  4%|▍         | 3/75 [25:11<4:27:40, 223.06s/it]


100%|██████████| 75/75 [07:37<00:00,  6.10s/it]t]

{'eval_loss': 0.32510998845100403, 'eval_rouge1': 0.8045117679255609, 'eval_rouge2': 0.7447073297073297, 'eval_rougeL': 0.7956091954022988, 'eval_rougeLsum': 0.7955610290093047, 'eval_runtime': 35.3232, 'eval_samples_per_second': 1.415, 'eval_steps_per_second': 0.368, 'epoch': 3.0}
{'train_runtime': 457.3417, 'train_samples_per_second': 1.312, 'train_steps_per_second': 0.164, 'train_loss': 0.5158934020996093, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=0.5158934020996093, metrics={'train_runtime': 457.3417, 'train_samples_per_second': 1.312, 'train_steps_per_second': 0.164, 'total_flos': 40405463138304.0, 'train_loss': 0.5158934020996093, 'epoch': 3.0})

In [13]:
# Save the trained model and tokenizer
model.save_pretrained('../models/t5')
tokenizer.save_pretrained('../models/t5')


('../models/t5\\tokenizer_config.json',
 '../models/t5\\special_tokens_map.json',
 '../models/t5\\spiece.model',
 '../models/t5\\added_tokens.json')